In [1]:
import os, re, sys
from datetime import date
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
import jpnotebooks.Software.SDLC_traceability_tools.user_tests_parser as utp
import scripts.tools.redmine_cache as rmc
from scripts.software_frs import frs_traceability2 as FRS

iie = importlib.reload(iie)
utp = importlib.reload(utp)
rmc = importlib.reload(rmc)

UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

In [2]:
_cache = None
_age = None
_client = None

def _get_client():
    global _client
    if _client is None:
        _client = rmc.RedmineClient()
    return _client

def _download_issues():
    _client = _get_client()
    return _client.get_filtered([('project.identifier', '==', 'pbssoftware')])

def load_issues():
    return _download_issues()


In [3]:
import re
class WebFRSIssuetrackerParser(iie.IssuetrackerParser):
    def __init__(self, types=("URS", "FRS", "SDS", "SWDS")):
        types = list(types) + ["3.0WebFRS"]
        super().__init__(types)
        
    def _get_result_for_line(self, line):
        """ identical to parent function, but checks the type after
        scanning the line to return only 3.0WebFRS items, converted
        to plain FRS items. 
        """
        if not line or line.isspace():
            return self._EMPTY_LINE, None, None, None, True

        m = self._item_match(line)
        if m is None: 
            return self._RAW_LINE, "", "", line.strip(), False
        
        dash1, typ, num, text, dash2 = m.groups()
        if typ != '3.0WebFRS':
            return self._EMPTY_LINE, None, None, None, True
        else:
            typ = 'FRS'
            num = "3." + num
        cancel = dash1 == dash2 and dash1 != ""
        return self._REQ_RESULT, typ, num, text, cancel

In [4]:
rtags = [
    "URS",
    "FRS",
    "SWDS",
    "CS",
    "BUG"
]

ttags = [
    "USR"
]

_ignore = {3194, 3287}
_sprints = {
    'Legacy',
    '3.0',
    '3.0.1',
    '3.0.2',
    '3.0.3',
    '3.0.4',
    '3.0.5',
    '3.0.6',
    '3.0.7'
}

def relevant(i):
    return i.sprint_milestone.name in _sprints and i.id not in _ignore and i.status != "Rejected"

In [19]:
iie = importlib.reload(iie)
utp = importlib.reload(utp)
UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

force = True
force = False
issues = load_issues()
issues2 = list(filter(relevant, issues.values()))
parser = iie.IssuetrackerParser(rtags)
reqs = parser.parse_all(issues2)

parser2 = UserTestsParser()
tests = parser2.parse_excel("C:\\Users\\Nathan\\Documents\\Dropbox\\FRS\\FRS Tests 190226.xlsx")

parser3 = WebFRSIssuetrackerParser(rtags)
reqs2 = parser3.parse_all(issues2)

allitems = reqs + tests + reqs2

def missing_parents(req, ref, reqs):
    if req.type == "TEST":
        return 'ignore'
    else:
        return 'fix'

rex = RequirementExtracter(rtags, ttags)
rex.config.missing_parents = missing_parents
rows = rex.extract(allitems)

In [26]:
from officelib.xllib import *
class JamesExcelFileRequirementsParser():
    def __init__(self, rtags):
        self.rtags = rtags
    
    def extract(self, fp):
        xl = Excel()
        with screen_lock(xl):
            wb = xl.Workbooks.Open(fp)
            ws = wb.Worksheets("specs list")
            reqs = self._extract(xl, wb, ws)
            wb.Close(False)
        return reqs
    
    def _extract(self, xl, wb, ws):
        cells = ws.Cells
        cr = cells.Range
        c1 = cr("B1")
        c2 = c1.End(xlc.xlDown).GetOffset(0, 4)
        data = cr(c1.GetOffset(1, 0), c2).Value2
        return self._get_data(data)
    
    def _get_data(self, data):
        out = []
        for num, text, refs, uprefs, iss in data:
            if num.startswith("<"):
                continue
            refs = [s.strip() for s in (refs or "").replace(",", "").split()]
            ref = iie.Reference("FRS", num, False, refs)
            out.append(ref)
        return out

In [27]:
newitems = JamesExcelFileRequirementsParser(rtags).extract("C:\\Users\\Nathan\\Documents\\Personal\\test\\NewPumpFeatures.xlsx")

In [22]:
import networkx as nx

import itertools

def impacted_nodes(G, node):
    a = nx.shortest_path_length(G, target=node)
    d = nx.shortest_path_length(G, source=node)
    return {n for n in itertools.chain(a, d)}

def impact_graph(G, node):
    r = impacted_nodes(G, node)
    return G.subgraph(r)

def items_to_graph(items):
    G = nx.DiGraph()
    for req in items:
        if not req.obs:
            for ref in req.refs:
                G.add_edge(ref, req.tag)
    return G

def dictify(items):
    out = {}
    for i in items:
        if not i.obs:
            if i.tag in out:
                print("Warning: duplicate for tag: %s"%i.tag)
            out[i.tag] = i
    return out

def add_implicit_local_refs(items):
    for req in items:
        hpref = req.tag.rsplit(".", 1)[0]
        if hpref != req.tag:
            req.refs.add(hpref)
            
def make_impact_graph(G, node, file):
    SG = impact_graph(G, node)
    nx.write_graphml(G, file)

In [30]:
FRS = importlib.reload(FRS)
_imkey = re.compile(r"(%s)(\d+)\.?([\d\.]*)" % "|".join(['URS', 'FRS', 'SDS'])).match

def key_match(key):
    m = _imkey(key)
    if m:
        type, first, others = m.groups()
        return type+first, others
    return key, ""

def root_for_type(items, typ):
    root = FRS.Root(key_match)
    for req in items:
        if req.type == typ and not req.obs:
            root.add(req.tag, 0)
    return root

In [84]:
add_implicit_local_refs(allitems)
add_implicit_local_refs(newitems)
urs = root_for_type(allitems, "URS")
G = items_to_graph(allitems)
reqs = dictify(allitems)

with open("test.nnf", 'w') as f:
    for u in urs.iter():
        if u.is_leaf():
            desc = nx.shortest_path_length(G, source=u.id)
            G2 = condensed_graph(desc, reqs)
            f.write("topN SWDev xx Req:%s\n"%(u.id))
            for a,b in G2.edges():
                f.write("Req:%s %s xx %s\n"%(u.id, a, b))
            

URS3652.7 {'URS3652'}


In [81]:
reqs["URS3652.7"]

False

In [10]:
testitems = []
def add(typ, num, refs=()):
    testitems.append(iie.Reference(typ, num, False, refs))
    
add("URS", "4370")
add("URS", "4370.6")
add("URS", "4370.6.2")
add("FRS", "4401")
add("FRS", "4401.2")
add("FRS", "4401.2.3", ["URS4370.6.2"])
add("FRS", "2321", ["FRS4401.2.3", "URS4370.6.2"])
add("FRS", "2321.1", ["URS4370.6.2"])
add("FRS", "2321.2", ["URS4370.6.2"])
add("FRS", "2321.3", ["URS4370.6.2"])
add("FRS", "2321.4", ["URS4370.6.2"])
add("FRS", "2321.5", ["URS4370.6.2"])

In [72]:
add_implicit_local_refs(allitems)
G = items_to_graph(allitems)
reqs = dictify(allitems)

def condensed_graph(impacted, reqs):

    # Impacted is a set of all nodes impacted, but has no edge data
    # Edge data is contained in G

    G2 = nx.DiGraph()

    # "fold" references into horizontal hierarchy whereever 
    # a parent of the same requirement type has the same URS as 
    # its children

    for node in impacted:
        req = reqs[node]
        hiparent = req.tag.rsplit(".", 1)[0]
        if hiparent == req.tag or hiparent not in impacted:
            # no parents to this req. just add refs
            for ref in req.refs:
                if ref not in impacted:
                    continue
                G2.add_edge(ref, req.tag)
        else:
            # maybe has refs to fold
            parent = reqs[hiparent]
            for ref in req.refs:
                if ref not in impacted:
                    continue
                if ref in parent.refs:
                    G2.add_edge(parent.tag, req.tag)
                else:
                    G2.add_edge(ref, req.tag)
    return G2

def condensed_impact_graph(G, epicenter, reqs):
    impacted = impacted_nodes(G, epicenter)
    # use a different color for epicenter
    G.add_node(epicenter, color="yellow")
    return condensed_graph(impacted, reqs)
    
# G2 = condensed_impact_graph(G, "FRS4401.2.3")
# nx.write_graphml(G2, "test.xml")

In [32]:
def condensed_implement_graph(G, node, reqs):
    d = nx.shortest_path_length(G, source=node)
    return condensed_graph(d, reqs)

In [71]:
def combined_implement_graph(G, nodes, reqs):
    d = set()
    for n in nodes:
        try:
            nrefs = impacted_nodes(G, n)
        except nx.NodeNotFound:
            pass
        else:
            d.update(nrefs)
    print(d)
    return condensed_graph(d, reqs)

In [45]:
target = "URS4370.6.2"
G2 = condensed_implement_graph(G, target, reqs)
nx.write_graphml(G2, "test.xml")
with open("colors.csv", 'w') as f:
    for node in G2.nodes():
        if node == target:
            color = "yellow"
        else:
            color="blue"
        f.write("%s,%s\n"%(node, color))

In [73]:

new = {n.tag for n in newitems}

for a in allitems:
    if a.tag in new:
        a.obs = True

apn = allitems + newitems
G = items_to_graph(apn)
reqs = dictify(apn)

G2 = combined_implement_graph(G, new, reqs)

nx.write_graphml(G2, "test.xml")

{'FRS4370.1.2', 'FRS4400.4.2', 'FRS4400.4.1', 'FRS3664.8.3', 'URS3664.1', 'FRS3664.4.9.1', 'URS3664.5', 'FRS4400.4.4.1', 'FRS4400.1.2.1', 'URS3664.3', 'FRS3664.4.8.1', 'FRS4401.2.13', 'FRS3664.13', 'FRS4370.1.1', 'FRS4400.3.2.1', 'FRS4401.2.13.2', 'FRS4400.4.4', 'FRS4401.2.15.1', 'FRS3664.12.2', 'FRS3664.7.5', 'FRS4400.1.1', 'FRS3664.8.1', 'FRS3664.7.6', 'FRS3664.4.9.2', 'FRS4400.3.4', 'FRS3664.13.2', 'FRS4400.2.3.1', 'FRS4401.2.15.2', 'FRS4400.2.1.1', 'FRS4400.2.1', 'FRS4400.1.4.1', 'FRS4400.3.1.1', 'FRS4400.1.4', 'FRS4400.3.3', 'FRS4401.2.12.1', 'FRS4370.1.2.2', 'FRS4401.2.11.2', 'URS4407.3', 'FRS3664.4.10.1', 'FRS4400.3.2', 'FRS4400.4.3.1', 'TEST2151', 'FRS3664.12.1', 'FRS4370.1.4', 'FRS4370.2.1', 'FRS4370.2.2', 'FRS4400.3.1', 'URS4370.11', 'FRS4401.2.11', 'FRS4370.1', 'FRS4400.1.1.1', 'FRS4400.2.3', 'FRS4400.1.3.1', 'FRS4401.2.12', 'FRS4370.1.3', 'URS4407', 'FRS4400.2.4', 'FRS4401.2.10.1', 'URS4370.12', 'FRS4370.2', 'URS3664.2', 'FRS3664.12', 'URS3664.6', 'FRS4400.2.2.1', 'FRS4401.

KeyError: 'FRS4400.4.2'

In [66]:
new

{'FRS3664.12',
 'FRS3664.12.1',
 'FRS3664.12.2',
 'FRS3664.13',
 'FRS3664.13.1',
 'FRS3664.13.2',
 'FRS3664.4.10',
 'FRS3664.4.10.1',
 'FRS3664.4.8.1',
 'FRS3664.4.9',
 'FRS3664.4.9.1',
 'FRS3664.4.9.2',
 'FRS3664.7.5',
 'FRS3664.7.6',
 'FRS3664.8.1',
 'FRS3664.8.3',
 'FRS4370.1',
 'FRS4370.1.1',
 'FRS4370.1.2',
 'FRS4370.1.2.1',
 'FRS4370.1.2.2',
 'FRS4370.1.3',
 'FRS4370.1.4',
 'FRS4370.1.5',
 'FRS4370.2',
 'FRS4370.2.1',
 'FRS4370.2.2',
 'FRS4401.2.10',
 'FRS4401.2.10.1',
 'FRS4401.2.10.2',
 'FRS4401.2.11',
 'FRS4401.2.11.1',
 'FRS4401.2.11.2',
 'FRS4401.2.11.3',
 'FRS4401.2.12',
 'FRS4401.2.12.1',
 'FRS4401.2.12.2',
 'FRS4401.2.12.3',
 'FRS4401.2.13',
 'FRS4401.2.13.1',
 'FRS4401.2.13.2',
 'FRS4401.2.13.3',
 'FRS4401.2.14',
 'FRS4401.2.14.1',
 'FRS4401.2.14.2',
 'FRS4401.2.14.3',
 'FRS4401.2.15',
 'FRS4401.2.15.1',
 'FRS4401.2.15.2',
 'FRS4401.2.3.1',
 'FRSURS4370.11',
 'FRSURS4370.12'}